<a href="https://colab.research.google.com/github/cheong0522/KBSC/blob/main/KBSC_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import math

In [ ]:
!pip install soynlp emoji

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 8.8 MB/s 
     |████████████████████████████████| 6.6 MB 39.5 MB/s 
     |████████████████████████████████| 120 kB 67.5 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import torch
from torch.nn import functional as F
from torch.utils.data import DataLoader, Dataset
from transformers import AutoTokenizer, AutoModel, AdamW 
import numpy as np
from tqdm.notebook import tqdm

In [ ]:
device = torch.device("cuda")

In [ ]:
test_data = pd.read_excel('/content/drive/MyDrive/KBSC_data.xlsx')

In [ ]:
test_data.loc[(test_data['Emotion'] == "행복"), 'Emotion'] = 0 
test_data.loc[(test_data['Emotion'] == "슬픔"), 'Emotion'] = 1  
test_data.loc[(test_data['Emotion'] == "분노"), 'Emotion'] = 2  
test_data.loc[(test_data['Emotion'] == "불안"), 'Emotion'] = 3  
test_data.loc[(test_data['Emotion'] == "중립"), 'Emotion'] = 4  

In [ ]:
train_dataset = []
for sen, label in zip(test_data['Sentence'], test_data['Emotion']):
  data_train = []
  data_train.append(sen)
  data_train.append(str(label))

  train_dataset.append(data_train)

In [ ]:
class TrainDataset(Dataset):
  
  def __init__(self, dataset):
    self.tokenizer = AutoTokenizer.from_pretrained("beomi/KcELECTRA-base")

    self.sentences = [str([i[0]]) for i in dataset]
    self.labels = [np.int32(i[1]) for i in dataset]

  def __len__(self):
    return (len(self.labels))
  
  def __getitem__(self, i):
    text = self.sentences[i]
    y = self.labels[i]

    inputs = self.tokenizer(
        text, 
        return_tensors='pt',
        truncation=True,
        max_length=64,
        pad_to_max_length=True,
        add_special_tokens=True
        )
    
    input_ids = inputs['input_ids'][0]
    attention_mask = inputs['attention_mask'][0]

    return input_ids, attention_mask, y

In [ ]:
train_dataset = TrainDataset(train_dataset)

In [ ]:
from torch import nn

model = AutoModel.from_pretrained("beomi/KcELECTRA-base", num_labels=5)
model = model.to(device)

In [ ]:
batch_size = 32
epochs = 1

In [ ]:
optimizer = AdamW(model.parameters(), lr=3e-5)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=5, shuffle=True)

In [ ]:
losses = []
accuracies = []

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

loss_fn = nn.CrossEntropyLoss()

for i in range(epochs):
  train_acc = 0.0
  total_loss = 0.0
  correct = 0
  total = 0
  batches = 0

  model.train()

  for input_ids_batch, attention_masks_batch, y_batch in tqdm(train_dataloader):
    optimizer.zero_grad()
    y_batch = y_batch.long().to(device)
    y_pred = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
    y_pred = y_pred[:, -1, :]
    loss = loss_fn(y_pred, y_batch)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    train_acc += calc_accuracy(y_pred, y_batch)
    total += len(y_batch)

    batches += 1
    if batches % 50 == 0:
      print("epoch {} loss {} train acc {}".format(i+1, loss.data.cpu().numpy(), train_acc / (batches+1)))
  print("epoch {} loss {} train acc {}".format(i+1, loss.data.cpu().numpy(), train_acc / (batches+1)))
  
  model.eval()

In [ ]:
def predict(sentence):
    data = [sentence, '0']
    dataset_another = [data]
    logits = 0
    another_test = TrainDataset(dataset_another)
    test_dataloader = torch.utils.data.DataLoader(another_test)

    model.eval()

    for input_ids_batch, attention_masks_batch, y_batch in test_dataloader:
        y_batch = y_batch.long().to(device)
        out = model(input_ids_batch.to(device), attention_mask=attention_masks_batch.to(device))[0]
        out = out[:, -1, :]

        for i in out:
            logits = i
            logits = logits.detach().cpu().numpy()
            logits = np.argmax(logits)
    return logits

predict("짜증나!")

In [ ]:
import socket
import time

In [ ]:
host = '127.0.01' # 호스트 ip를 적어주세요
port = 8080            # 포트번호를 임의로 설정해주세요

In [ ]:
server_sock = socket.socket(socket.AF_INET)
server_sock.bind((host, port))
server_sock.listen(1)
print("기다리는 중..")

In [ ]:
while True: #안드로이드에서 연결 버튼 누를 때까지 기다림
    client_sock, addr = server_sock.accept() # 연결 승인

    if client_sock: #client_sock 가 null 값이 아니라면 (연결 승인 되었다면)

        print('Connected by?!', addr) #연결주소 print
        in_data = client_sock.recv(1024) #안드로이드에서 "refresh" 전송
        print('rcv :', in_data.decode("utf-8"), len(in_data)) #전송 받은값 디코딩

        #밑에 in_data를 일기 내용이라고 생각하고
        predict(in_data)

        import random
        num = random.randint(1,30)
        print(num)

        if emotion == 0 :
          happy_music_data = pd.read_excel('/content/drive/MyDrive/happy_music_data.xlsx')
          out_data = happy_music_data.loc[num]
        elif emotion == 1 :
          sad_music_data = pd.read_excel('/content/drive/MyDrive/sad_music_data.xlsx')
          out_data = sad_music_data.loc[num]
        elif emotion == 2 :
          anger_music_data = pd.read_excel('/content/drive/MyDrive/anger_music_data.xlsx')
          out_data = anger_music_data.loc[num]
        elif emotion == 3 :
          nervous_music_data = pd.read_excel('/content/drive/MyDrive/nervous_music_data.xlsx')
          out_data = nervous_music_data.loc[num]
        elif emotion == 4 :
          neutrality_music_data = pd.read_excel('/content/drive/MyDrive/neutrality_music_data.xlsx')
          out_data = neutrality_music_data.loc[num]

        while in_data : #2초마다 안드로이드에 값을 전달함 (추후 , STOP , Connect 옵션 설정 가능)
            client_sock.send(str(out_data).encode("utf-8")) # string 으로 인코딩해서 전송, byte 로 전송하면 복잡함
           
            print('send :', out_data) #아웃 데이터에 넣어야함 노래 제목이나 감정 수치 넣어야함

            #서버에 2초마다 데이터 전송
            time.sleep(2)

In [ ]:
client_sock.close()
server_sock.close()